In [1]:
from typing_extensions import TypedDict, List, Optional, Union,Annotated,Sequence
from dotenv import load_dotenv
from langchain_core.messages import BaseMessage,ToolMessage,SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph,END
from langgraph.prebuilt import  ToolNode
load_dotenv()

True

In [2]:
def supervisor(state:StateGraph)-> StateGraph:
    """
    This function is a placeholder for the supervisor functionality.
    It currently does nothing but can be expanded in the future.
    """
    return 


In [3]:
def medicine_agent(state:StateGraph) -> StateGraph:
    """
    This function is a placeholder for the medicine agent functionality.
    It currently does nothing but can be expanded in the future.
    """
    return state    

In [4]:
def appointment_scheduler(state:StateGraph) -> StateGraph:
    """
    This function is a placeholder for the appointment scheduler functionality.
    It currently does nothing but can be expanded in the future.
    """
    return state

In [5]:
def case_generator(state:StateGraph) -> StateGraph:
    """
    This function is a placeholder for the case generator functionality.
    It currently does nothing but can be expanded in the future.
    """
    return state    

In [6]:
def dataverification(state:StateGraph) -> StateGraph:
    """
    This function is a placeholder for the data verification functionality.
    It currently does nothing but can be expanded in the future.
    """
    return state    

In [7]:
workflow = StateGraph(
    name="StateGraph",
    description="A graph representing the state of the system.",
    initial_state=state,
    final_states=[next_state],
    on_enter=on_enter,
    on_exit=on_exit
)

NameError: name 'state' is not defined

### Learning

In [8]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage],add_messages]   
    

In [9]:
@tool
def add(a:int,b:int):
    """This tools add two numbers a and b"""
    return a + b


In [10]:
tools = [add]

In [11]:
model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    ).bind_tools(tools)

In [12]:
from langchain_core.messages import HumanMessage, AIMessage

In [13]:
def model_call(state:AgentState) -> AgentState:
    system_prompt = SystemMessage(
        content="You are a helpful assistant that answers questions based on the provided messages and tools"
    )
    # Initialize the state with the system message if not already present
    response = model.invoke([system_prompt]+ state["messages"])
    return {"messages": [response]}

In [14]:
def should_continue(state:AgentState):
    message = state['messages']
    last_message = message[-1] 

    if not last_message.tool_calls:
        return "end"
    else:
        return "continue"

In [15]:
graph = StateGraph(AgentState)
graph.add_node("our_agent", model_call)

tool_node = ToolNode(tools = tools)
graph.add_node("tools",tool_node)
graph.set_entry_point("our_agent")

graph.add_conditional_edges("our_agent",should_continue,
                            {
                                "continue": "tools",
                                "end": END
                            })

graph.add_edge("tools", "our_agent")

In [16]:
app = graph.compile()

In [17]:
def print_stream(stream):
    for s in stream:
        message = s['messages'][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [18]:
inputs = {
    "messages": [("user","add 3+22, tell me a joke")]
}
print_stream(app.stream(inputs, stream_mode="values"))

================================ Human Message =================================

add 3+22, tell me a joke


================================== Ai Message ==================================
Tool Calls:
  add (b8801d66-a336-4f7e-9ff4-425f6369fe1e)
 Call ID: b8801d66-a336-4f7e-9ff4-425f6369fe1e
  Args:
    b: 22.0
    a: 3.0
================================= Tool Message =================================
Name: add

25
================================== Ai Message ==================================

Why don't scientists trust atoms? 

Because they make up everything!


# Graph IV
- Agent for creating documents and verify from the human, when the  verification is done the flow will terminate/exit 
    * after the execution of the tool , the tool will not send the toolMessage to the llm

In [19]:
document_content = ""

In [20]:
@tool
def update(content: str) -> str:
    """
    updates the document content with the provided content.
    """
    global document_content
    document_content = content + "\n"
    return f"Document updated successfully.The current content is:\n{document_content}"

In [21]:
@tool
def save(filename: str) -> str:
    """
    Saves the current document content to a text file and finish the process.
    Args:
        filename (str): The name of the file to save the content to.
    """
    global document_content

    # Ensure document_content is defined
    if "document_content" not in globals():
        document_content = ""

    if not filename.endswith('.txt'):
        filename = f'{filename}.txt'
    try:
        with open(filename, 'w') as file:
            print("--------------------------------------inside save function ----------------------------------------------------")
            print(f"\n{document_content}\n")
            print("--------------------------------------inside save function ----------------------------------------------------")
            file.write(document_content)
        print(f"\n Document saved to {filename}")
        return f"Document saved to {filename}"
    except Exception as e:
        return f"Error saving document: {str(e)}"



In [22]:
tools1 = [ update, save]

In [23]:
%pip install google-genai



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [24]:
from google import genai

client = genai.Client()

print("List of models that support generateContent:\n")
for m in client.models.list():
    for action in m.supported_actions:
        if action == "generateContent":
            print(m.name)

print("List of models that support embedContent:\n")
for m in client.models.list():
    for action in m.supported_actions:
        if action == "embedContent":
            print(m.name)

List of models that support generateContent:

models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gem

In [25]:
model1 = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite",
    ).bind_tools(tools1)

In [26]:
def doc_agent(state:AgentState) -> AgentState:
    
    system_prompt = SystemMessage( 
        content=f"""
        You are a Drafter, a helpful writing assistant. You are going to help the user update and save documents based on the provided messages and tools
            - If the user wants to update or modify content,use the 'update' tool with the complete updated content.
            - If the user wants to save and finish , you need to use the 'save' tool.
            - make sure to always show the current document state after modifications.
            - here are your tools  {tools1}
            
            The current docuement content is:{document_content}
            """
    )
    if not state["messages"]:
        user_input = "I'm ready to help you update a document. What would you like to create? "
        user_message = HumanMessage(content=user_input)


    else:
        user_input = input("\nWhat would you like to do with the document?")
        print(f"\nUser: {user_input}")
        user_message = HumanMessage(content=user_input)

    all_messages = list(state["messages"]) + [user_message] + [system_prompt]
    response = model1.invoke(all_messages)


    print(f"\nAssistant: {response.content}")
    # Only check tool_calls if response is an AIMessage
    if isinstance(response, AIMessage) and getattr(response, 'tool_calls', None):
        print(f"Using tools:{[tc['name'] for tc in response.tool_calls]}")
    return {"messages": list(state["messages"]) + [user_message, response]}

In [27]:
def should_continue_doc(state:AgentState) -> str:
    """ Determines whether we should continue or end the conversation."""
    messages = state['messages']
    if not messages:
        return "continue"
    
    for message in reversed(messages):
        if (
            isinstance(message, ToolMessage)
            and "saved" in message.content.lower()# type: ignore
            and "document" in message.content.lower()# type: ignore
        ):
            return "end"
        
    return "continue"


In [28]:
def print_messages(messages):

    """Function I made  to print the messages in t a more readable format"""

    if not messages:
        print("No messages to display.")
        return

    for message in messages[-3:]:
        if isinstance(message, ToolMessage):
            print(f"\n TOOL RESULT: {message.content}")
  

In [29]:
graph1 = StateGraph(AgentState)

graph1.add_node("doc_agent", doc_agent)
graph1.add_node("tools",ToolNode(tools=tools1))

graph1.set_entry_point("doc_agent")
graph1.add_edge("doc_agent", "tools")

graph1.add_conditional_edges("tools",
                              should_continue_doc,
                            {
                                "continue": "doc_agent",
                                "end": END
                            },)

In [30]:
app1 = graph1.compile()

In [31]:
def run_document_agent():
    print("\n ================== Document Agent ================\n")
    
    state = {"messages": []}

    for step in app1.stream(state, stream_mode="values"):
        if "messages" in step:
            print_messages(step["messages"])
    print("\n======================DRAFTER FINISHED=======================.")

In [32]:
run_document_agent()


 ================== Document Agent ================

No messages to display.

Assistant: Okay, I'm ready. Please provide the content for the document.

User: work from home

Assistant: 
Using tools:['update']

 TOOL RESULT: Document updated successfully.The current content is:
work from home


User: update

Assistant: Okay. Is there anything else you would like to add or modify?

 TOOL RESULT: Document updated successfully.The current content is:
work from home


 TOOL RESULT: Document updated successfully.The current content is:
work from home


User: add the work flie balance

Assistant: 
Using tools:['update']

 TOOL RESULT: Document updated successfully.The current content is:
work from home
work flie balance


User: save

Assistant: OK. What would you like to name the file?

 TOOL RESULT: Document updated successfully.The current content is:
work from home
work flie balance


 TOOL RESULT: Document updated successfully.The current content is:
work from home
work flie balance


Us

/home/tejas.raval@simform.dom/Desktop/ViniGraph/pheonix/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:1568: UserWarning: HumanMessage with empty content was removed to prevent API error
  warnings.warn(



Assistant: 


KeyboardInterrupt: Interrupted by user

# Graph V